 * trailer: 
     * show a histogram with examples of distance / orientation with samples over ALL datasets
     * show perf!  show time to fetch frames using Spark + Parquet
     * show a video of one camera with debug overlays.  maybe one with delauny lidar too (!)
     * show a frame HTML with 3d interface
     * show new things: argo associated bikes, delauny lidar, occlusion tree
 * supported datasets, how to get a blurb and **stats** on each of them.  prolly render histo reports for each.
 * data structures:
    * StampedDatum
    * Frame

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('/opt/psegs')

In [4]:
from psegs.datasets import nuscenes

nuscenes.NuscStampedDatumTableLabelsAllFrames.build()




# from psegs.datasets import kitti

# kitti.DSUtil.test()

2021-01-27 08:18:49,856	ps   20306 : Going to write in 284 chunks ...
2021-01-27 08:21:18,780	oarph 20306 : Progress for 
save_df_thunks [Pid:20306 Id:140308222907392]
-----------------------  --------------------------------------
Thruput
N thru                   1 (of 284)
N chunks                 1
Total time               2 minutes and 28.92 seconds
Total thru               2.5 GB
Rate                     16.81 MB / sec
Hz                       0.006715205276215152
Progress
Percent Complete         0.352112676056338
Est. Time To Completion  11 hours, 42 minutes and 23.16 seconds
-----------------------  --------------------------------------
2021-01-27 08:24:13,260	oarph 20306 : Progress for 
save_df_thunks [Pid:20306 Id:140308222907392]
-----------------------  --------------------------------------------------------------------------------
Thruput
N thru                   2 (of 284)
N chunks                 2
Total time               5 minutes and 23.39 seconds
Total thru        

2021-01-27 08:42:52,802	oarph 20306 : Progress for 
save_df_thunks [Pid:20306 Id:140308222907392]
-----------------------  --------------------------------------------------------------------------------
Thruput
N thru                   9 (of 284)
N chunks                 9
Total time               24 minutes and 2.89 seconds
Total thru               22.19 GB
Rate                     15.38 MB / sec
Hz                       0.006237467960955823
Progress
Percent Complete         3.1690140845070425
Est. Time To Completion  12 hours, 14 minutes and 48.4 seconds
Latency (per chunk)
Avg                      2 minutes, 40 seconds, 321 milliseconds, 464 microseconds and 776.99 nanoseconds
p50                      2 minutes, 28 seconds, 915 milliseconds, 775 microseconds and 299.07 nanoseconds
p95                      3 minutes, 18 seconds, 424 milliseconds, 583 microseconds and 339.69 nanoseconds
p99                      3 minutes, 27 seconds, 262 milliseconds, 723 microseconds and 903.66 nano

2021-01-27 09:01:29,106	oarph 20306 : Progress for 
save_df_thunks [Pid:20306 Id:140308222907392]
-----------------------  -------------------------------------------------------------------------------
Thruput
N thru                   17 (of 284)
N chunks                 17
Total time               42 minutes and 39.16 seconds
Total thru               39.93 GB
Rate                     15.6 MB / sec
Hz                       0.006642811707770986
Progress
Percent Complete         5.985915492957746
Est. Time To Completion  11 hours, 9 minutes and 53.82 seconds
Latency (per chunk)
Avg                      2 minutes, 30 seconds, 538 milliseconds, 664 microseconds and 46.46 nanoseconds
p50                      2 minutes, 25 seconds, 69 milliseconds, 356 microseconds and 918.33 nanoseconds
p95                      3 minutes, 7 seconds, 376 milliseconds, 907 microseconds and 634.74 nanoseconds
p99                      3 minutes, 25 seconds, 53 milliseconds, 188 microseconds and 762.66 nanoseco

2021-01-27 09:22:44,948	oarph 20306 : Progress for 
save_df_thunks [Pid:20306 Id:140308222907392]
-----------------------  --------------------------------------------------------------------------------
Thruput
N thru                   25 (of 284)
N chunks                 25
Total time               1 hour, 3 minutes and 54.96 seconds
Total thru               58.42 GB
Rate                     15.23 MB / sec
Hz                       0.006518973341646534
Progress
Percent Complete         8.80281690140845
Est. Time To Completion  11 hours, 2 minutes and 10.18 seconds
Latency (per chunk)
Avg                      2 minutes, 33 seconds, 398 milliseconds, 387 microseconds and 689.59 nanoseconds
p50                      2 minutes, 27 seconds, 238 milliseconds, 497 microseconds and 257.23 nanoseconds
p95                      3 minutes, 23 seconds, 948 milliseconds, 421 microseconds and 192.17 nanoseconds
p99                      3 minutes, 37 seconds, 99 milliseconds, 36 microseconds and 140.4

2021-01-27 09:43:13,547	oarph 20306 : Progress for 
save_df_thunks [Pid:20306 Id:140308222907392]
-----------------------  --------------------------------------------------------------------------------
Thruput
N thru                   33 (of 284)
N chunks                 33
Total time               1 hour, 24 minutes and 23.52 seconds
Total thru               76.73 GB
Rate                     15.15 MB / sec
Hz                       0.006517207721381421
Progress
Percent Complete         11.619718309859154
Est. Time To Completion  10 hours, 41 minutes and 53.43 seconds
Latency (per chunk)
Avg                      2 minutes, 33 seconds, 439 milliseconds, 945 microseconds and 871.18 nanoseconds
p50                      2 minutes, 28 seconds, 658 milliseconds, 356 microseconds and 666.56 nanoseconds
p95                      3 minutes, 14 seconds, 395 milliseconds, 723 microseconds and 390.58 nanoseconds
p99                      3 minutes, 36 seconds, 296 milliseconds, 217 microseconds and

...

2021-01-27 18:49:36,548	oarph 20306 : Progress for 
save_df_thunks [Pid:20306 Id:140308222907392]
-----------------------  --------------------------------------------------------------------------------
Thruput
N thru                   249 (of 284)
N chunks                 249
Total time               10 hours, 30 minutes and 45.42 seconds
Total thru               562.63 GB
Rate                     14.87 MB / sec
Hz                       0.006579396463385954
Progress
Percent Complete         87.67605633802818
Est. Time To Completion  1 hour, 28 minutes and 39.64 seconds
Latency (per chunk)
Avg                      2 minutes, 31 seconds, 989 milliseconds, 624 microseconds and 818.17 nanoseconds
p50                      2 minutes, 28 seconds, 414 milliseconds, 690 microseconds and 17.7 nanoseconds
p95                      3 minutes, 8 seconds, 135 milliseconds, 14 microseconds and 104.84 nanoseconds
p99                      3 minutes, 32 seconds, 933 milliseconds, 842 microseconds and 2

2021-01-27 19:10:11,998	oarph 20306 : Progress for 
save_df_thunks [Pid:20306 Id:140308222907392]
-----------------------  --------------------------------------------------------------------------------
Thruput
N thru                   257 (of 284)
N chunks                 257
Total time               10 hours, 51 minutes and 20.81 seconds
Total thru               580.93 GB
Rate                     14.86 MB / sec
Hz                       0.006576116793862499
Progress
Percent Complete         90.49295774647888
Est. Time To Completion  1 hour, 8 minutes and 25.77 seconds
Latency (per chunk)
Avg                      2 minutes, 32 seconds, 65 milliseconds, 425 microseconds and 743.85 nanoseconds
p50                      2 minutes, 28 seconds, 462 milliseconds, 256 microseconds and 431.58 nanoseconds
p95                      3 minutes, 7 seconds, 329 milliseconds, 845 microseconds and 762.25 nanoseconds
p99                      3 minutes, 32 seconds, 401 milliseconds, 291 microseconds and 

2021-01-27 19:30:19,164	oarph 20306 : Progress for 
save_df_thunks [Pid:20306 Id:140308222907392]
-----------------------  --------------------------------------------------------------------------------
Thruput
N thru                   265 (of 284)
N chunks                 265
Total time               11 hours, 11 minutes and 27.94 seconds
Total thru               599.12 GB
Rate                     14.87 MB / sec
Hz                       0.006577650579131981
Progress
Percent Complete         93.30985915492958
Est. Time To Completion  48 minutes and 8.57 seconds
Latency (per chunk)
Avg                      2 minutes, 32 seconds, 29 milliseconds, 966 microseconds and 926.57 nanoseconds
p50                      2 minutes, 28 seconds, 462 milliseconds, 256 microseconds and 431.58 nanoseconds
p95                      3 minutes, 6 seconds, 897 milliseconds, 323 microseconds and 560.71 nanoseconds
p99                      3 minutes, 31 seconds, 868 milliseconds, 739 microseconds and 757.54 n

2021-01-27 19:51:55,999	oarph 20306 : Progress for 
save_df_thunks [Pid:20306 Id:140308222907392]
-----------------------  --------------------------------------------------------------------------------
Thruput
N thru                   273 (of 284)
N chunks                 273
Total time               11 hours, 33 minutes and 4.73 seconds
Total thru               618.52 GB
Rate                     14.87 MB / sec
Hz                       0.006564909238162018
Progress
Percent Complete         96.12676056338029
Est. Time To Completion  27 minutes and 55.58 seconds
Latency (per chunk)
Avg                      2 minutes, 32 seconds, 325 milliseconds, 30 microseconds and 510.24 nanoseconds
p50                      2 minutes, 28 seconds, 658 milliseconds, 356 microseconds and 666.56 nanoseconds
p95                      3 minutes, 7 seconds, 732 milliseconds, 429 microseconds and 933.55 nanoseconds
p99                      3 minutes, 31 seconds, 336 milliseconds, 188 microseconds and 488.01 n

2021-01-27 20:11:40,091	oarph 20306 : Progress for 
save_df_thunks [Pid:20306 Id:140308222907392]
-----------------------  --------------------------------------------------------------------------------
Thruput
N thru                   281 (of 284)
N chunks                 281
Total time               11 hours, 52 minutes and 48.79 seconds
Total thru               635.75 GB
Rate                     14.86 MB / sec
Hz                       0.006570212262855682
Progress
Percent Complete         98.94366197183099
Est. Time To Completion  7 minutes and 36.61 seconds
Latency (per chunk)
Avg                      2 minutes, 32 seconds, 202 milliseconds, 84 microseconds and 193.45 nanoseconds
p50                      2 minutes, 28 seconds, 701 milliseconds, 332 microseconds and 330.7 nanoseconds
p95                      3 minutes, 6 seconds, 927 milliseconds, 261 microseconds and 590.96 nanoseconds
p99                      3 minutes, 30 seconds, 803 milliseconds, 637 microseconds and 218.48 na

In [1]:
import sys
sys.path.append('/opt/psegs')
from oarphpy.spark import NBSpark
NBSpark.SRC_ROOT = '/opt/psegs'
NBSpark.SRC_ROOT_MODULES = ['psegs']
NBSpark.MAYBE_REBUILD_EGG_EVERY_CELL_RUN = False
NBSpark.CONF_KV = {
    'spark.driver.memory': '8g',
    'spark.pyspark.python': 'python3',
    'spark.python.worker.reuse': False,
    'spark.sql.files.maxPartitionBytes': int(8 * 1e6),
    'spark.port.maxRetries': '256',
  }
from psegs.datasets import kitti

spark = NBSpark.getOrCreate()
df = kitti.KITTISDTable.as_df(spark)
# df = spark.read.parquet('/tmp/avs_test/test_kitti_sd_table_tracking/sd_table/')
df.show(5)
# print(df.count())

2020-09-27 22:49:30,378	oarph 40 : Using source root /opt/psegs 
2020-09-27 22:49:30,379	oarph 40 : Using source root /opt 
2020-09-27 22:49:30,547	oarph 40 : Generating egg to /tmp/tmpb7yp4q0b_oarphpy_eggbuild ...
2020-09-27 22:49:30,561	oarph 40 : ... done.  Egg at /tmp/tmpb7yp4q0b_oarphpy_eggbuild/psegs-0.0.0-py3.8.egg
2020-09-27 22:49:34,166	ps   40 : Loading /opt/psegs/dataroot/stamped_datum/stamped_datums ...


AnalysisException: Path does not exist: file:/opt/psegs/dataroot/stamped_datum/stamped_datums;

In [9]:
# seg = df.filter('uri.segment_id = "kitti-tracking-train-0009"')

In [10]:
# seg_uri = seg.select('uri').persist()
df.createOrReplaceTempView('data')
# seg_uri.show()

In [11]:
spark.sql("""
    SELECT
      uri.segment_id AS seg, 
      uri.topic AS topic,
      count(*) AS N,
      MAX(uri.timestamp),
      MIN(uri.timestamp),
      (MAX(uri.timestamp) - MIN(uri.timestamp)) * 1e-9 AS len
    FROM data
    GROUP BY topic, seg
    ORDER BY len ASC
""").show(1000, truncate=False)

+-------------------------+--------+----+---------------------------------+---------------------------------+-------------+
|seg                      |topic   |N   |max(uri.timestamp AS `timestamp`)|min(uri.timestamp AS `timestamp`)|len          |
+-------------------------+--------+----+---------------------------------+---------------------------------+-------------+
|kitti-tracking-train-0012|ego_pose|78  |1317047690486674021              |1317047682506437291              |7.980236730  |
|kitti-tracking-test-0027 |ego_pose|85  |9400000000                       |1000000000                       |8.400000000  |
|kitti-tracking-train-0014|ego_pose|106 |1317212788870919845              |1317212777989915167              |10.881004678 |
|kitti-tracking-test-0006 |ego_pose|114 |12300000000                      |1000000000                       |11.300000000 |
|kitti-tracking-test-0001 |ego_pose|147 |15600000000                      |1000000000                       |14.600000000 |
|kitti-t